In [1]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2
import cv2.aruco as aruco
from google.colab.patches import cv2_imshow # Import cv2_imshow

# Load your image (replace with your image path or use a frame from a video)
image = cv2.imread("/content/drive/My Drive/pagetrack/Detect_ArUco_marker.png")

# Convert image to grayscale (required for detection)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Define the dictionary of markers you used
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

# Initialize the detector parameters using default values
parameters = aruco.DetectorParameters()

# Create the detector
detector = aruco.ArucoDetector(aruco_dict, parameters)

# Detect the markers in the image
corners, ids, rejected = detector.detectMarkers(gray)

# Draw detected markers on the image
if ids is not None:
    print("Detected marker IDs:", ids.flatten())
    aruco.drawDetectedMarkers(image, corners, ids)
else:
    print("No markers detected.")

# Show the result using cv2_imshow
cv2_imshow(image)

# Remove cv2.waitKey(0) and cv2.destroyAllWindows() as they are not needed with cv2_imshow
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Output hidden; open in https://colab.research.google.com to view.

In [25]:
!pip install transformers datasets torch torchvision
!pip install opencv-python

In [26]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch

# Load model and processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

# Load image
image_path = "/content/drive/My Drive/pagetrack/OCR_test.jpg"
image = Image.open(image_path).convert("RGB")

# Prepare image
pixel_values = processor(images=image, return_tensors="pt").pixel_values

# Inference
generated_ids = model.generate(pixel_values)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Save to file
with open("/content/drive/My Drive/pagetrack/handwriting_output.txt", "w") as f:
    f.write(generated_text)

print("✅ OCR done. Output saved to handwriting_output.txt")

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

✅ OCR done. Output saved to handwriting_output.txt


In [27]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image
import torch

# Load model & processor
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")
model.eval()  # <-- important for consistent results

# Load & preprocess image
image = Image.open("/content/drive/My Drive/pagetrack/OCR_test.jpg").convert("RGB")

# Optional: upscale if handwriting is small
# image = image.resize((image.width * 2, image.height * 2))

# Prepare inputs
pixel_values = processor(images=image, return_tensors="pt").pixel_values

# Generate output
with torch.no_grad():  # important for inference
    generated_ids = model.generate(pixel_values)

# Decode output
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

# Save to file
with open("/content/drive/My Drive/pagetrack/handwriting_output.txt", "w") as f:
    f.write(generated_text)

print("✅ Result:", generated_text)

Config of the encoder: <class 'transformers.models.vit.modeling_vit.ViTModel'> is overwritten by shared encoder config: ViTConfig {
  "attention_probs_dropout_prob": 0.0,
  "encoder_stride": 16,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "model_type": "vit",
  "num_attention_heads": 12,
  "num_channels": 3,
  "num_hidden_layers": 12,
  "patch_size": 16,
  "pooler_act": "tanh",
  "pooler_output_size": 768,
  "qkv_bias": false,
  "torch_dtype": "float32",
  "transformers_version": "4.51.3"
}

Config of the decoder: <class 'transformers.models.trocr.modeling_trocr.TrOCRForCausalLM'> is overwritten by shared decoder config: TrOCRConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.0,
  "cross_attention_hidden_size": 768,
  "d_mod

✅ Result: 0 0


In [29]:
image.show()  # Make sure the image is readable, not cropped or rotated

In [7]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# Load image
img_path = "/content/drive/My Drive/pagetrack/OCR_test.jpg"  # Upload your image to Colab first
image = cv2.imread(img_path)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Optional: improve contrast
gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# Save preprocessed image
cv2.imwrite("/content/drive/My Drive/pagetrack/processed.png", gray)

True